In [1]:
%matplotlib inline

In [3]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib import rcParams

import numpy as np
import pandas as pd


import shapely.geometry as sgeom

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

import geopandas as gpd


paper_path = Path('../../paper/figures/')

slide_path = Path('../../slides/figures/intro/')
proposal_path = Path('../../draft/figures/intro/')
#rcParams['font.family'] = 'Segoe Print'

In [4]:
YEAR = 2021
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

In [5]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [6]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys())).dropna(subset=['STATE'])

In [7]:
nydf = df[df['STATE'].str.match("NY")]

In [10]:
ds['ELEMENT'].unique().compute()

0     TMAX
1     TMIN
2     PRCP
3     TAVG
4     SNWD
      ... 
59    TSUN
60    DWPR
61    WT07
62    SN36
63    WT10
Name: ELEMENT, Length: 64, dtype: object

In [12]:
nyds = ds[ds['ID'].isin(nydf['ID'].unique()) & ds['ELEMENT'].isin(['TAVG', 'PRCP' ])].compute()

In [13]:
ny = nyds.merge(nydf, on='ID')

In [21]:
ny.columns

Index(['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG',
       'OBS-TIME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME',
       'GSN FLAG', 'HCN/CRN FLAG', 'WMO ID'],
      dtype='object')

In [10]:
ds['ELEMENT'].unique().compute()

0     TMAX
1     TMIN
2     PRCP
3     TAVG
4     SNWD
      ... 
59    TSUN
60    DWPR
61    WT07
62    SN36
63    WT10
Name: ELEMENT, Length: 64, dtype: object

In [36]:
nyd = ny.pivot(index=['NAME','LATITUDE', 'LONGITUDE', 'DATE'], columns=['ELEMENT'], values=['DATA_VALUE']).reset_index()

In [44]:
[c for i, c in nyd.columns.to_flat_index()]

['', '', '', '', 'PRCP', 'TAVG']

In [14]:
ny.to_csv("nydata", index=False)

In [35]:
ny.pivot?